In [ ]:
import requests
from bs4 import BeautifulSoup as BS
import unicodedata
import re
import pandas as pd
import numpy as np
import datetime
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
df_launch = pd.read_csv('launch.csv')
df_f9 = df_launch[df_launch['BoosterVersion'] == 'Falcon 9'].copy()
df_f9.loc[:,'FlightNumber'] = list(range(1, df_f9.shape[0]+1))
df_f9['PayloadMass'].replace(pd.NA, df_f9['PayloadMass'].mean(), inplace=True)
df_f9.to_csv('f9.csv', index=False)

In [ ]:
def date_time(table_cells):
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    

In [ ]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"
response = requests.get(static_url)

In [ ]:
soup = BS(response.content, 'html.parser')
html_tables = soup.find_all('table')
first_launch_table = html_tables[2]

column_names = []
rows = first_launch_table.find_all('th')
for row in rows:
    column_name = extract_column_from_header(row)
    column_names.append(column_name)

In [ ]:
launch_dict= dict.fromkeys(column_names)

del launch_dict['Date and time ( )']
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

In [ ]:
extracted_row = 0

for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
    for rows in table.find_all("tr"):
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False
        row=rows.find_all('td')
        if flag:
            extracted_row += 1
            launch_dict['Flight No.'].append(flight_number)
            datatimelist=date_time(row[0])
            
            date = datatimelist[0].strip(',')
            launch_dict['Date'].append(date)
            
            time = datatimelist[1]
            launch_dict['Time'].append(time)
            
            bv=booster_version(row[1])
            if not(bv):
                bv=row[1].a.string
            launch_dict['Version Booster'].append(bv)
            
            launch_site = row[2].a.string
            launch_dict['Launch site'].append(launch_site)
            
            payload = row[3].a.string
            launch_dict['Payload'].append(payload)
            
            payload_mass = get_mass(row[4])
            launch_dict['Payload mass'].append(payload_mass)
            
            orbit = row[5].a.string
            launch_dict['Orbit'].append(orbit)
            
            customer = row[6].a.string if row[6].a else row[6].get_text(strip=True)
            launch_dict['Customer'].append(customer)
            
            launch_outcome = list(row[7].strings)[0]
            launch_dict['Launch outcome'].append(launch_outcome)
            
            booster_landing = landing_status(row[8])
            launch_dict['Booster landing'].append(booster_landing)

In [ ]:
df_launch2 = pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })
df_launch2.to_csv('launch2.csv', index=False)